In [1]:
#This notebook will be used mainly for the capstone project
#
import pandas as pd
import numpy as np
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Clearly define data science problem and identify stakeholders
INTRODUCTION / BUSINESS PROBLEM

The problem that we will be aiming to figure out, through creating a model, if there is a way to advise drivers of potential future accidents by analyzing a combination of variables and predicting the collision severity. From the Data Collisions file provided, several potential indicators of an accident include Weather, Road conditions, and Light conditions. This model will be used to assist drivers and hopefully assist in detering accidents when driving conditions are less than ideal. I believe there is a need for solving this type of problem as often times it is unclear whether or not conditions for driving in certain areas lead to higher accident probability, and therefore collision severity. 

# Describe the data you will be using to solve the problem
DATA

The data comes in a csv file with lots of columns that will not be used in our model. There are few variables(columns) that I believe will influence the model which include the Light Conditions(LIGHTCOND), the Road conditions(ROADCOND), and the Weather(WEATHER). The target column is Severity Code(SEVERITYCODE). Therefore the first step is going to be cleaning the data and selecting for the aforementioned columns. The next step will be to transform the values in some of the columns to respective integer values. For example, in respect to the column Road conditions, the values are 'Wet', 'Dry', and 'unknown' each of which can be mapped to a respective integer value of 1, 2, and 0. The same sort of methodology can be followed to map the values from Light Conditions, and Weather to numerical values.


In [4]:
df = pd.read_csv('Data-Collisions.csv') #read in the csv
df.head() #show first 5 entries

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


As we can see there are 38 columns and clearly we do not need all of these so we will begin to clear the columns that are not necessary.

In [5]:
#initialize array with entries to drop
to_drop = ['X','Y', 'OBJECTID', 'INCKEY', 'COLDETKEY',
          'REPORTNO', 'STATUS', 'ADDRTYPE', 'INTKEY',
          'LOCATION', 'EXCEPTRSNCODE', 'EXCEPTRSNDESC',
          'PEDCOUNT', 'PEDCYLCOUNT', 'INCDATE', 'INCDTTM',
          'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC', 
          'INATTENTIONIND', 'PEDROWNOTGRNT', 'SDOTCOLNUM', 
           'ST_COLCODE','ST_COLDESC', 'SEGLANEKEY', 'CROSSWALKKEY',
          'HITPARKEDCAR']

df.drop(to_drop,inplace=True,axis=1)

In [6]:
df.head()

,SEVERITYCODE,SEVERITYCODE.1,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,VEHCOUNT,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,SPEEDING
0,2,2,Injury Collision,Angles,2,2,N,Overcast,Wet,Daylight,NaN
1,1,1,Property Damage Only Collision,Sideswipe,2,2,0,Raining,Wet,Dark - Street Lights On,NaN
2,1,1,Property Damage Only Collision,Parked Car,4,3,0,Overcast,Dry,Daylight,NaN
3,1,1,Property Damage Only Collision,Other,3,3,N,Clear,Dry,Daylight,NaN
4,2,2,Injury Collision,Angles,2,2,0,Raining,Wet,Daylight,NaN


This looks a lot better. There is a column called UNDERINF, which refers to drivers under the influence. This has values that could take on either a character value of 'Y' and 'N', as well as an integer value that can be 0 or greater. We want to remove rows where this column value is either 'Y', or has a value greater than 0.

In [12]:
to_drop= df[df['UNDERINFL'] == 'Y'].index
df.drop(to_drop, inplace=True)

In [14]:
df.head()
to_drop= df[df['UNDERINFL'] == '1'].index
df.drop(to_drop, inplace=True)
df.head()

,SEVERITYCODE,SEVERITYCODE.1,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,VEHCOUNT,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,SPEEDING
0,2,2,Injury Collision,Angles,2,2,N,Overcast,Wet,Daylight,NaN
1,1,1,Property Damage Only Collision,Sideswipe,2,2,0,Raining,Wet,Dark - Street Lights On,NaN
2,1,1,Property Damage Only Collision,Parked Car,4,3,0,Overcast,Dry,Daylight,NaN
3,1,1,Property Damage Only Collision,Other,3,3,N,Clear,Dry,Daylight,NaN
4,2,2,Injury Collision,Angles,2,2,0,Raining,Wet,Daylight,NaN


Now that we are sure that the entries for drivers in our dataframe were not under the influence, we can go ahead and get rid of the column entirely. Additionally I am going to get rid of another column that has all NaN values. 

In [16]:
to_drop = ['UNDERINFL', 'SPEEDING']
df.drop(to_drop, inplace=True, axis=1)
df.head()

,SEVERITYCODE,SEVERITYCODE.1,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,VEHCOUNT,WEATHER,ROADCOND,LIGHTCOND
0,2,2,Injury Collision,Angles,2,2,Overcast,Wet,Daylight
1,1,1,Property Damage Only Collision,Sideswipe,2,2,Raining,Wet,Dark - Street Lights On
2,1,1,Property Damage Only Collision,Parked Car,4,3,Overcast,Dry,Daylight
3,1,1,Property Damage Only Collision,Other,3,3,Clear,Dry,Daylight
4,2,2,Injury Collision,Angles,2,2,Raining,Wet,Daylight


In [17]:
to_drop = ['COLLISIONTYPE']
df.drop(to_drop, inplace=True, axis=1)
df.head()

,SEVERITYCODE,SEVERITYCODE.1,SEVERITYDESC,PERSONCOUNT,VEHCOUNT,WEATHER,ROADCOND,LIGHTCOND
0,2,2,Injury Collision,2,2,Overcast,Wet,Daylight
1,1,1,Property Damage Only Collision,2,2,Raining,Wet,Dark - Street Lights On
2,1,1,Property Damage Only Collision,4,3,Overcast,Dry,Daylight
3,1,1,Property Damage Only Collision,3,3,Clear,Dry,Daylight
4,2,2,Injury Collision,2,2,Raining,Wet,Daylight


In [18]:
to_drop = ['SEVERITYCODE.1']
df.drop(to_drop, inplace=True, axis=1)
df.head()

,SEVERITYCODE,SEVERITYDESC,PERSONCOUNT,VEHCOUNT,WEATHER,ROADCOND,LIGHTCOND
0,2,Injury Collision,2,2,Overcast,Wet,Daylight
1,1,Property Damage Only Collision,2,2,Raining,Wet,Dark - Street Lights On
2,1,Property Damage Only Collision,4,3,Overcast,Dry,Daylight
3,1,Property Damage Only Collision,3,3,Clear,Dry,Daylight
4,2,Injury Collision,2,2,Raining,Wet,Daylight


Now we can see the data looks a lot different than it did when we began, and is in a form much closer to one that can be used for training our model. 